In [ ]:
%run main --yaml_path yamls/protein_vec.yaml

In [2]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_8.yaml


-----Load Model-----



Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForTripletSimilarity(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 320, padding_idx=1)
      (dropout): Dropout(p=0.05, inplace=False)
      (position_embeddings): Embedding(1026, 320, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-5): 6 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=320, out_features=320, bias=True)
              (key): Linear(in_features=320, out_features=320, bias=True)
              (value): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.05, inplace=False)
            )
            (LayerNorm): LayerNorm((320,), eps=1e-05, el

Generating valid split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/49667 [00:00<?, ? examples/s]

...

  0%|          | 0/524 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/517 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/517 [00:00<?, ?it/s]

In [3]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_35.yaml


-----Load Model-----



Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForTripletSimilarity(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.05, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-11): 12 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.05, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, 

Evaluating sim metrics:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

...

  0%|          | 0/1048 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/1034 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/1034 [00:00<?, ?it/s]

In [4]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_150.yaml


-----Load Model-----



Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForTripletSimilarity(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 640, padding_idx=1)
      (dropout): Dropout(p=0.05, inplace=False)
      (position_embeddings): Embedding(1026, 640, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-29): 30 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=640, out_features=640, bias=True)
              (key): Linear(in_features=640, out_features=640, bias=True)
              (value): Linear(in_features=640, out_features=640, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=640, out_features=640, bias=True)
              (dropout): Dropout(p=0.05, inplace=False)
            )
            (LayerNorm): LayerNorm((640,), eps=1e-05, 

Evaluating sim metrics:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/197 [00:00<?, ?it/s]

  0%|          | 0/2064 [00:00<?, ?it/s]

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_650.yaml

In [ ]:
import torch
from transformers import BertConfig, BertTokenizer
from models.model_zoo import *
from utils import get_yaml
from glob import glob

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
yargs = get_yaml('yamls/SE/cvd.yaml')
config = BertConfig.from_pretrained(yargs['general_args']['model_path'])
for key, value in yargs['general_args'].items(): # copy yaml config into args
    setattr(config, key, value)


In [ ]:
weights = [path.replace('\\', '/') for path in glob('./weights/*.pt')]
weights

In [ ]:
paths = []
for weight in weights:
    model = torch.load(weight)
    try:
        model.config = model.bert.config
    except:
        print(weight)
        continue
    path = 'lhallee/' + weight.split('/')[-1].split('.pt')[0]
    paths.append(path)
    #model.push_to_hub(path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('results/NLP_MOE')

In [ ]:
for path in paths:
    tokenizer.push_to_hub(path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('lhallee/sci_moe_all')

In [ ]:
tokenizer.push_to_hub('lhallee/all_nlp_singlemoe_MI')